# Inference using SYCL backend on AI PC

## Introduction

This notebook demonstrates how to install LLamacpp for SYCL on Windows with Intel GPUs. It applies to Intel Core Ultra and Core 11 - 14 gen integrated GPUs (iGPUs), as well as Intel Arc Series GPU.

## What is an AIPC

What is an AI PC you ask?

Here is an [explanation](https://www.intel.com/content/www/us/en/newsroom/news/what-is-an-ai-pc.htm#gs.a55so1) from Intel:

”An AI PC has a CPU, a GPU and an NPU, each with specific AI acceleration capabilities. An NPU, or neural processing unit, is a specialized accelerator that handles artificial intelligence (AI) and machine learning (ML) tasks right on your PC instead of sending data to be processed in the cloud. The GPU and CPU can also process these workloads, but the NPU is especially good at low-power AI calculations. The AI PC represents a fundamental shift in how our computers operate. It is not a solution for a problem that didn’t exist before. Instead, it promises to be a huge improvement for everyday PC usages.”

## Install Prerequisites

### Step 1: System Preparation

To set up your AIPC for running with Intel iGPUs, follow these essential steps:

1. Update Intel GPU Drivers: Ensure your system has the latest Intel GPU drivers, which are crucial for optimal performance and compatibility. You can download these directly from Intel's [official website](https://www.intel.com/content/www/us/en/download/785597/intel-arc-iris-xe-graphics-windows.html) . Once you have installed the official drivers, you could also install Intel ARC Control to monitor the gpu:

   <img src="Assets/gpu_arc_control.png">


2. Install Visual Studio 2022 Community edition with C++: Visual Studio 2022, along with the “Desktop Development with C++” workload, is required. This prepares your environment for C++ based extensions used by the intel SYCL backend that powers accelerated Ollama. You can download VS 2022 Community edition from the official site, [here](https://visualstudio.microsoft.com/downloads/).

3. Install conda-forge: conda-forge will manage your Python environments and dependencies efficiently, providing a clean, minimal base for your Python setup. Visit conda-forge's [installation site](https://conda-forge.org/download/) to install for windows.

4. Install [Intel® oneAPI Base Toolkit](https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html)

   

## Step 2: Install Llamacpp for SYCL
The llama.cpp SYCL backend is designed to support Intel GPU firstly. Based on the cross-platform feature of SYCL.

### After installation of conda-forge, open the Miniforge Prompt, and create a new python environment:
  ```
  conda create -n llm-sycl python=3.11

  ```

### Activate the new environment
```
conda activate llm-sycl

```

<img src="Assets/llm4.png">

### With the llm-sycl environment active, enable oneAPI environment. 
Type oneapi in the windows search and then open the Intel oneAPI command prompt for Intel 64 for Visual Studio 2022 App.

<img src="Assets/oneapi1.png">

#### Run the below command in the VS command prompt and you should see the below sycl devices displayed in the console
There should be one or more level-zero GPU devices displayed as ext_oneapi_level_zero:gpu.

```
sycl-ls

```

<img src="Assets/oneapi2.png">

### Install build tools

* Download & install [cmake for Windows](https://cmake.org/download/):
* The new Visual Studio will install Ninja as default. (If not, please install it manually: https://ninja-build.org/)

### Install llama.cpp

* git clone the llama.cpp repo
  
  ```
  git clone https://github.com/ggerganov/llama.cpp.git

  ```
  
* On the oneAPI command line window, step into the llama.cpp main directory and run the following:
  
  ```
  @call "C:\Program Files (x86)\Intel\oneAPI\setvars.bat" intel64 --force

    # Option 1: Use FP32 (recommended for better performance in most cases)
  cmake -B build -G "Ninja" -DGGML_SYCL=ON -DCMAKE_C_COMPILER=cl -DCMAKE_CXX_COMPILER=icx  -DCMAKE_BUILD_TYPE=Release
    
    # Option 2: Or FP16
  cmake -B build -G "Ninja" -DGGML_SYCL=ON -DCMAKE_C_COMPILER=cl -DCMAKE_CXX_COMPILER=icx  -DCMAKE_BUILD_TYPE=Release -DGGML_SYCL_F16=ON
    
    cmake --build build --config Release -j

  ```

### Below shows a simple example to show how to run a community GGUF model with llama.cpp for SYCL
* Download the model from huggingface and prepare the model for inference
* Run the model for example as below
* Open the mini-forge prompt, activate the llm-sycl environment and enable oneAPI enviroment as below

  ```
  "C:\Program Files (x86)\Intel\oneAPI\setvars.bat" intel64  
  ```
* List the sycl devices as below

  ```
  build\bin\ls-sycl-device.exe

  ```
* Run inference
```
build\bin\llama-cli.exe -m models\llama-2-7b.Q4_0.gguf -p "Building a website can be done in 10 simple steps:\nStep 1:" -n 400 -e -ngl 33 -s 0 -sm none -mg 0
```

<img src="Assets/cmd1.png">

### Below is an example output

<img src="Assets/out1.png">



## Run the inference

In [ ]:
! ..\git_llamacpp\llama.cpp\build\bin\llama-cli.exe -m Qwen1.5-4B.Q4_K_M.gguf -p "Building a website can be done in 10 simple steps:\nStep 1:" -n 400 -e -ngl 25 -s 0 -sm none -mg 0

#### Run the inference locally on AI PC

In [ ]:
! ..\git_llamacpp\llama.cpp\build\bin\llama-cli.exe -m ./download_models/{model_gguf} -p "Building a website can be done in 10 simple steps:\nStep 1:" -n 100 -e -ngl 33 -s 0 -sm none -mg 0

## Example output

<img src="Assets/output_latest.png">

* Reference:https://github.com/ggerganov/llama.cpp/blob/master/docs/backend/SYCL.md